In [1]:
import modin.pandas as dd
import json
import pandas as pd
import numpy as np
import os
import string
import re
from fuzzywuzzy import fuzz

In [2]:
f=open('cluster2.txt')
contents=f.read()

In [3]:
contents=contents.replace('[',"").replace("]","").replace("'","")
contents=contents.split(',')

In [4]:
filelist=[c.split('.')[0].replace(" ","") for c in contents]
columns=[c.split('.')[1] for c in contents]

In [5]:
f=os.listdir('NYCColumns/')

In [6]:
def is_vehicle_type(s):
    vehicles=['AGC AGRICULTURE', 'AGR AGRICULTURAL', 'AMB AMBULANCE', 'ARG AIR NATIONAL GUARD', 'ATD ATV DEALER', 
          'ATV ALL-TERRAIN VEHICLE', 'AYG ARMY NATIONAL GUARD', 'BOB BIRTHPLACE OF BASEBALL', 'BOT MOTORBOATS', 
          'CBS COUNTY BOARD OF LEGISLATORS', 'CCK COUNTY CLERK', 'CHC COMM. HS/HOLD CARRIER', 'CLG COUNTY LEGISLATORS', 
          'CME CORONER/MEDICAL EXAMINER', 'CMH CONGRESSIONAL MEDAL-OF-HONOR', 'COM COMMERCIAL', 'CSP SPORTS COMMERCIAL', 
          'DLR DEALER', 'EDU EDUCATOR', 'FAR FARM', 'FPW FORMER PRISONER OF WAR', 'GAC GOVERNOR ADDITIONAL CAR', 
          'GSC GOVERNOR SECOND CAR', 'GSM GOLD STAR MOTHERS', 'HAC HAM OPERATOR COMMERCIAL', 'HAM HAM OPERATOR', 
          'HIF HEARSE INV. SPEC. FEE', 'HIR HEARSE/HEARSE INVALID', 'HIS HISTORICAL', 'HOU HOUSE/COACH TRAILER', 
          'HSM HISTORICAL MOTORCYCLE', 'IRP INTERNATIONAL REGISTRATION PLAN', 'ITP INTRANSIT PERMIT', 
          'JCA JUSTICE CRT. OF APPEALS', 'JCL JUSTICE CRT. OF CLAIMS', 'JSC JSC APPELLATE DIV.', 
          'JWV JEWISH WAR VETERANS', 'LMA LIMITED USE MCY C1 A', 'LMB LIMITED USE MCY C1 B', 
          'LMC LIMITED USE MCY C1 C', 'LOC LOCOMOTIVE', 'LTR LIGHT TRAILER', 'LUA LIMITED USE AUTOMOBILE', 
          'MCD MOTORCYCLE DEALER', 'MCL MARINE CORP LEAGUE', 'MED MEDICAL DOCTOR', 'MOT MOTORCYCLE', 
          'NLM NAVAL MILITIA', 'NYA NEW YORK ASSEMBLY', 'NYC NEW YORK COUNCIL', 'NYS NEW YORK SENATE', 
          'OMF OMNIBUS FRANCHISE BUS', 'OML OMNIBUS LIVERY', 'OMO OMNIBUS OUT OF STATE', 'OMR OMNIBUS REGULAR', 
          'OMS OMNIBUS PRIVATE RENTAL', 'OMT OMNIBUS TAXI', 'OMV VANITY OMNIBUS', 'ORC COMMERCIAL ORGANIZATIONS', 
          'ORG ORGANIZATIONS', 'ORM ORGANIZATION MOTORCYCLE', 'PAS PASSENGER', 'PHS PEARL HARBOR SURVIVORS', 
          'PPH PURPLE HEART', 'PSD POLITICAL SUBDIVISION', 'RGC REGIONAL COMMERCIAL', 'RGL REGIONAL', 
          'SCL SCHOOL CAR', 'SEM SEMI-TRAILER', 'SNO SNOWMOBILE', 'SOS SURVIVORS OF THE SHIELD', 
          'SPC SPECIAL PURPOSE COMM.', 'SPO SPORTS', 'SRF SPECIAL REGISTRATION FEE', 'SRN', 'SPECIAL REGISTRATION', 
          '(JUDGES/OFFICIALS)', 'STA STATE OWNED VEHICLE', 'STG STATE NATIONAL GUARD', 'SUP JUSTICE SUPREME COURT', 
          'THC TRACTOR HOUSE/HOLD CARRIER', 'TOW TOW TRUCK', 'TRA TRANSPORTER', 'TRC TRACTOR (REGULAR)', 
          'TRL TRAILER (REGULAR)', 'USC US CONGRESS', 'USS US SENATE', 'VAS VOLUNTEER AMBULANCE SERVICES', 
          'VPL VAN POOL', 'WUG WORLD UNIVERSITY GAMES', 'FIRE FIRE TRUCK', 'CONV CONVERTIBLE', 'SEDN SEDAN', 
          'SUBN SUBURBAN', '4DSD FOUR-DOOR SEDAN', '2DSD TWO-DOOR SEDAN', 'H/WH HOUSE ON WHEELS', 
          'ATV ALL TERRAIN VEHICLE', 'MCY MOTORCYCLE', 'H/IN HEARSE-INVALID', 'LOCO LOCOMOTIVE', 'CUST CUSTOM', 
          'RPLC REPLICA', 'AMBU AMBULANCE', 'P/SH POWER SHOVEL', 'RBM ROAD BUILDING MACHINE', 'R/RD ROAD ROLLER', 
          'RD/S ROAD SWEEPER', 'S/SP', 'SAND OR AGRICULTRAL', 'SPREADER/SPRAYER', 'SN/P SNOW PLOW', 'TRAV SNOW TRAVELER', 
          'MOBL SNOWMOBILE', 'TR/E TRACTION ENGINE', 'T/CR TRACTOR CRANE', 'TR/C TRUCK CRANE', 'SWT TRUCK W/SMALL WHEELS', 
          'W/DR WELL DRILLER', 'W/SR WELL SERVICING RIG', 'FPM FEED PROCESSING MACHINE', 'MCC MOBILE CAR CRUSHER', 
          'EMVR EARTH MOVER', 'TRAC TRACTOR', 'N/A NOT APPLICABLE', 'DELV DELIVERY TRUCK', 'DUMP DUMP TRUCK', 
          'FLAT FLAT BED TRUCK', 'PICK PICK-UP TRUCK', 'STAK STAKE TRUCK', 'TANK TANK TRUCK', 'REFG REFRIGERATOR TRAILER',
          'TOW TOW TRUCK', 'VAN VAN TRUCK', 'UTIL UTILITY', 'POLE POLE TRAILER', 'BOAT BOAT', 'H/TR HOUSE TRAILER', 
          'SEMI SEMI-TRAILER', 'TRLR TRAILER', 'LTRL LIGHT TRAILER', 'LSVT LOW SPEED VEHICLE - TRUCK', 'BUS BUS(OMNIBUS)',
          'LIM LIMOUSINE(OMNIBUS)', 'HRSE HEARSE(AMBULANCE)', 'TAXI TAXI', 'DCOM DISABLED COMMERICAL', 'CMIX CEMENT MIXER',
          'MOPD MOPED', 'MFH MANUFACTURED HOME', 'SNOW SNOWMOBILE', 'LSV LOW SPEED VEHICLE']
    
    
    x=s
    if type(s)!=str or s==None:
        return 0
    is_vehicle=0
    for v in vehicles:
        ratio=fuzz.partial_ratio(v[0].lower(), x.lower())
        if ratio>=85:
            is_vehicle=1
    return is_vehicle



In [7]:
def is_building_classification(s):
    if type(s)!=str or s==None:
        return 0
    
    building_classifications=pd.read_html('https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html')[0]
    codes=list(building_classifications['Building Code'])
    description=list(building_classifications['Description'])

    for i,desc in enumerate(description):
        if len(desc)==1:
            description.remove(desc)
        if len(codes[i])==1:
            codes.remove(codes[i])

    building_classifications=[]

    for i, code in enumerate(codes):
        building_classifications.append(code + '-'+description[i])
    
    
    is_build_class=0
    for Class in building_classifications:
        ratio=fuzz.partial_ratio(Class.lower(), s.lower())
        if ratio>=85:
            is_build_class=1
    return is_build_class

In [8]:
def is_location_type(s):
    
    location_types=['accounting', 'airport', 'amusement_park', 'aquarium', 'art_gallery', 'atm', 'bakery', 'bank', 
                'bar', 'beauty_salon', 'bicycle_store', 'book_store', 'bowling_alley', 'bus_station', 'cafe', 
                'campground', 'car_dealer', 'car_rental', 'car_repair', 'car_wash', 'casino', 'cemetery', 'church',
                'city_hall', 'clothing_store', 'convenience_store', 'courthouse', 'dentist', 'department_store',
                'doctor', 'drugstore', 'electrician', 'electronics_store', 'embassy', 'fire_station', 'florist',
                'funeral_home', 'furniture_store', 'gas_station', 'grocery_or_supermarket', 'gym', 'hair_care',
                'hardware_store', 'hindu_temple', 'home_goods_store', 'hospital', 'insurance_agency', 
                'jewelry_store', 'laundry', 'lawyer', 'library', 'light_rail_station', 'liquor_store',
                'local_government_office', 'locksmith', 'lodging', 'meal_delivery', 'meal_takeaway', 'mosque',
                'movie_rental', 'movie_theater', 'moving_company', 'museum', 'night_club', 'painter', 'park',
                'parking', 'pet_store', 'pharmacy', 'physiotherapist', 'plumber', 'police', 'post_office',
                'primary_school', 'real_estate_agency', 'restaurant', 'roofing_contractor', 'rv_park', 'school',
                'secondary_school', 'shoe_store', 'shopping_mall', 'spa', 'stadium', 'storage', 'store',
                'subway_station', 'supermarket', 'synagogue', 'taxi_stand', 'tourist_attraction', 'train_station',
                'transit_station', 'travel_agency', 'university', 'veterinary_care', 'zoo', 
                'administrative_area_level_1', 'administrative_area_level_2', 'administrative_area_level_3',
                'administrative_area_level_4', 'administrative_area_level_5', 'archipelago', 'colloquial_area',
                'continent', 'country', 'establishment', 'finance', 'floor', 'food', 'general_contractor',
                'geocode', 'health', 'intersection', 'locality', 'natural_feature', 'neighborhood', 
                'place_of_worship', 'point_of_interest', 'political', 'post_box', 'postal_code',
                'postal_code_prefix', 'postal_code_suffix', 'postal_town', 'premise', 'room', 'route',
                'street_address', 'street_number', 'sublocality', 'sublocality_level_1', 'sublocality_level_2', 
                'sublocality_level_3', 'sublocality_level_4', 'sublocality_level_5', 'subpremise', 'town_square',
                'ABANDONED BUILDING','AIRPORT TERMINAL', 'ATM', 'BANK', 'BAR/NIGHT CLUB', 'BEAUTY & NAIL SALON',
                'BOOK/CARD', 'BRIDGE', 'BUS (NYC TRANSIT)', 'BUS (OTHER)', 'BUS STOP', 'BUS TERMINAL',
                'CANDY STORE', 'CEMETERY', 'CHAIN STORE', 'CHECK CASHING BUSINESS', 'CHURCH', 
                'CLOTHING/BOUTIQUE', 'COMMERCIAL BUILDING', 'CONSTRUCTION SITE', 'DEPARTMENT STORE',
                'DOCTOR/DENTIST OFFICE', 'DRUG STORE', 'DRY CLEANER/LAUNDRY', 'FACTORY/WAREHOUSE', 'FAST FOOD',
                'FERRY/FERRY TERMINAL', 'FOOD SUPERMARKET', 'GAS STATION', 'GROCERY/BODEGA', 'GYM/FITNESS FACILITY',
                'HIGHWAY/PARKWAY', 'HOSPITAL', 'HOTEL/MOTEL', 'JEWELRY', 'LIQUOR STORE', 'LOAN COMPANY',
                'MAILBOX INSIDE', 'MAILBOX OUTSIDE', 'MARINA/PIER', 'MOSQUE', 'OPEN AREAS (OPEN LOTS)', 'OTHER',
                'OTHER HOUSE OF WORSHIP', 'PARK/PLAYGROUND', 'PARKING LOT/GARAGE (PRIVATE)',
                'PARKING LOT/GARAGE (PUBLIC)', 'PHOTO/COPY', 'PRIVATE/PAROCHIAL SCHOOL', 'PUBLIC BUILDING',
                'PUBLIC SCHOOL', 'RESIDENCE - APT. HOUSE', 'RESIDENCE - PUBLIC HOUSING', 'RESIDENCE-HOUSE',
                'RESTAURANT/DINER', 'SHOE', 'SMALL MERCHANT', 'SOCIAL CLUB/POLICY', 'STORAGE FACILITY', 
                'STORE UNCLASSIFIED', 'STREET', 'SYNAGOGUE', 'TAXI (LIVERY LICENSED)', 'TAXI (YELLOW LICENSED)', 
                'TAXI/LIVERY (UNLICENSED)', 'TELECOMM. STORE', 'TRAMWAY', 'TRANSIT - NYC SUBWAY', 
                'TRANSIT FACILITY (OTHER)', 'TUNNEL', 'VARIETY STORE', 'VIDEO STORE']
    
    if type(s)!=str or s==None:
        return 0
    is_loc_type=0
    for loc in location_types:
        ratio=fuzz.ratio(loc.lower(), s.lower())
        if ratio>=85:
            is_loc_type=1
    return is_loc_type

In [9]:
def is_park(s):
    parks=['Barretto Point Park', 'Bicentennial Veterans Memorial Park', 'Bridge Park', 'Bronx Park', 'New York Botanical Garden', 'Bronx Zoo', 'Castle Hill Park', 'Claremont Park', 'Concrete Plant Park', 'Crotona Park', 'Devoe Park', 'Estella Diggs Park', 'Ewen Park', 'Ferry Point Park', 'Givans Creek Woods', 'Grant Park', 'Haffen Park', 'Harding Park', 'Harris Park', 'Henry Hudson Park', 'Hunts Point Riverside Park', 'Jerome Park', 'Joseph Rodman Drake Park', 'Joyce Kilmer Park', 'Julius Richman Park', 'Macombs Dam Park', 'Mill Pond Park', 'Mullaly Park', 'Pelham Bay Park', 'Orchard Beach', "Rodman's Neck", 'Playground 52', 'Poe Park, including Edgar Allan Poe Cottage', "Printer's Park", 'Pugsley Creek Park', 'Rainey Park', 'Raoul Wallenberg Forest', 'Richman Park', 'Riverdale Park', 'Roberto Clemente State Park (NYSP)', 'Seton Falls Park', 'Seton Park', 'Soundview Park', 'Spuyten Duyvil Shorefront Park', 'St. James Park', "St. Mary's Park", 'Starlight Park', 'Tremont Park', 'University Woods', 'Van Cortlandt Park', 'Vidalia Park', 'Vinmont Veteran Park', "Washington's Walk", 'Wave Hill', 'Williamsbridge Oval', 'American Veterans Memorial Pier', 'Amersfort Park', 'Asser Levy Park', 'Bath Beach Park', 'Betsy Head Park', 'Bensonhurst Park', 'Breukelen Ballfields Park', 'Brooklyn Botanic Garden', 'Brooklyn Bridge Park', 'Brooklyn Heights Promenade', 'Brooklyn–Queens Greenway', 'Brooklyn Waterfront Greenway', 'Brower Park', 'Bushwick Inlet Park', 'Byrne Park', 'Cadman Plaza', 'Canarsie Park', 'Carroll Park', 'Calvert Vaux Park', 'City Line Park', 'Coffey Park', 'Columbus Park', 'Commodore Barry Park', 'Coney Island Beach & Boardwalk', 'Coney Island Creek Park', 'Continental Army Plaza', 'Cooper Park', 'Domino Park', 'Dyker Beach Park and Golf Course', 'East River State Park (NYSP)', 'Floyd Bennett Field (NPS - Gateway National Recreation Area)', 'Fort Greene Park', 'Four Sparrow Marsh', 'Frank M. Charles Memorial Park', 'Fresh Creek Nature Preserve', 'Friends Field', 'Fulton Park', 'Gravesend Park', 'Green Central Knoll', 'Herbert Von King Park', 'Highland Park', 'Irving Square Park', 'JJ Byrne Park', 'John J. Carty Park', 'John Paul Jones Park', 'Kaiser Park', 'Kelly Park', 'Leif Ericson Park', 'Leon S. Kaiser Playground', 'Lieutenant Joseph Petrosino Park', 'Lincoln Terrace Park', 'Linden Park', 'Lindower Park', 'Linton Park', 'Macri Triangle', 'Manhattan Beach Park', 'Maria Hernandez Park', 'Marine Park', 'McCarren Park', 'McGolrick Park', 'McKinley Park', 'Mount Prospect Park', "Owl's Head Park", 'Paerdegat Basin Park', 'Paerdegat Park', 'Prospect Park', 'Red Hook Park', 'Robert E. Venable Park', 'Saratoga Park', 'Scarangella Park', 'Seaside Park', 'Seth Low Park', 'Shore Park and Parkway', 'Spring Creek Park', 'Sternberg Park', 'St. Johns Park', 'Sunset Park', 'Tompkins Park', 'Trinity Park', 'Van Voorhees Park', 'Walt Whitman Park', 'Wingate Park', 'WNYC Transmitter Park', 'Zion Triangle', '79th Street Boat Basin', 'Abingdon Square Park', 'Albert Capsouto Park', 'Asphalt Green', 'The Battery', 'Bella Abzug (Hudson) Park', 'Bennett Park', 'Blackwell Island Light, Roosevelt Island', 'British Garden at Hanover Square', 'Bowling Green', 'Bryant Park', 'Captain Patrick J. Brown Walk', 'Carl Schurz Park', 'Central Park', 'Chelsea Park', 'City Hall Park', 'Collect Pond', 'Columbus Park', 'Corlears Hook', 'Dag Hammarskjold Plaza', 'Damrosch Park', 'Dante Park', 'DeSalvio Playground', 'DeWitt Clinton Park', 'Drumgoole Plaza', 'Duane Park', 'East River Greenway', 'East River Esplanade', 'East River Park', 'Stuyvesant Cove', 'Father Demo Square', 'Finn Square', 'Foley Square', 'Fort Tryon Park', 'Fort Washington Park', 'Franklin D. Roosevelt Four Freedoms Park (NYSP)', 'Gorman Park', 'Governors Island', 'Governors Island National Monument (NPS)', 'Gramercy Park', 'Greenacre Park', 'Hamilton Fish Park', 'Hanover Square', 'Harlem River Park', "Hell's Kitchen Park", 'Herald Square', 'High Line', 'Highbridge Park', 'Holcombe Rucker Park', 'Hudson River Park', 'Imagination Playground at Burling Slip', 'Inwood Hill Park', 'Isham Park', 'Jay Hood Wright Park', 'Jackie Robinson Park', 'Jackson Square Park', 'John Jay Park', 'Liberty Park', 'Louis Cuvillier Park', 'Madison Square', 'Manhattan Waterfront Greenway', 'Marcus Garvey Park', 'Mill Rock Park', 'Mitchell Square Park', 'Morningside Park', "Murphy's Brother's Playground", 'Muscota Marsh', 'Paley Park', 'Peretz Square', 'Peter Cooper Park', 'Plaza Lafayette', 'Queen Elizabeth II September 11th Garden', 'Ralph Bunche Park', 'Randalls and Wards Islands', 'Richard Tucker Square', 'Riverbank State Park (NYSP)', 'Riverside Park', 'Robert Moses Playground', 'Roosevelt Triangle', 'Rucker Park', 'Sakura Park', 'Samuel N. Bennerson 2nd Playground', 'Sara Delano Roosevelt Park', 'Septuagesimo Uno', 'Seward Park', 'Sheridan Square', 'Sherman Square', 'Southpoint Park, Roosevelt Island', 'St. Nicholas Park', 'St. Vartan Park', 'Straus Park', 'Stuyvesant Square', 'Swindler Cove Park', 'Teardrop Park', 'Theodore Roosevelt Park', 'Thomas Jefferson Park', 'Thomas Paine Park', 'Tompkins Square Park', 'Tribeca Park', 'Union Square', 'Verdi Square', 'Vesuvio Playground', 'Randalls and Wards Islands', 'Washington Market Park', 'Washington Square Park', 'West Side Community Garden', 'Winston Churchill Park', 'Zuccotti Park', 'Admiral Park', 'Alley Park', 'Alley Pond Park', 'Andrews Grove', 'Astoria Park', 'Baisley Pond Park', 'Bayside Fields', 'Bayswater Park', 'Bayswater Point State Park (NYSP)', 'Beach Channel Park', 'Big Bush Park', 'Bowne Park', 'Brant Point Wildlife Sanctuary', 'Breininger Park', 'Broad Channel American Park', 'Broad Channel Park', 'Broad Channel Wetlands', 'Brooklyn–Queens Greenway', 'Brookville Park', 'Bulova Park', 'Captain Tilly Park', 'Crocheron Park', 'Cunningham Park', 'Detective Keith L Williams Park', 'Detective William T. Gunn Park', 'Dr. Charles R. Drew Park', 'Dubos Point Wildlife Sanctuary', 'Doughboy Park', 'Douglaston Park', 'Elmhurst Park', 'Evergreen Park', 'Flushing Fields', 'Flushing Meadows–Corona Park', 'Forest Park', 'Fort Tilden (NPS - Gateway National Recreation Area)', 'Fort Totten', 'Francis Lewis Park', 'Frank Golden Park', 'Frank Principe Park', 'Gantry Plaza State Park (NYSP)', 'Haggerty Park', 'Harvey Park', 'Highland Park', 'Hinton Park', 'Hoffman Park', 'Hook Creek Park', "Hunter's Point Park", 'Idlewild Park', 'Jamaica Bay Park', 'Jamaica Bay Wildlife Refuge (NPS - Gateway National Recreation Area)', 'Jacob Riis Park (NPS - Gateway National Recreation Area)', 'John Golden Park', 'Juniper Valley Park', 'Kissena Park', 'Kohlreiter Square', 'Libra Triangle', 'Linnaeus Park', 'Little Bay Park', 'Louis Pasteur Park', 'Mafera Park', 'Macneil Park', 'Marconi Park', 'Marie Curie Park', 'Montbellier Park', "O'Donohue Park", 'Overlook Park', 'Park of the Americas', 'Phil "Scooter" Rizzuto', 'Police Officer Edward Byrne Park', "Powell's Cove Park", 'Queens Botanical Garden', 'Queens County Farm Museum', 'Queens Zoo', 'Queensbridge Park', 'Rachel Carson Playground', 'Railroad Park', 'Ralph Demarco Park', 'Rochdale Park', 'Rockaway Beach', 'Rockaway Community Park', 'Rufus King Park', 'Seagirt Avenue Wetlands', 'Socrates Sculpture Park', 'Spring Creek Park', 'Springfield Park', 'St. Albans Park', 'Travers Park', 'Triangle 54', 'Tudor Park', "Udall's Park Preserve", 'Wayanda Park', 'Aesop Park', 'Allison Pond Park', 'Amundsen Circle', 'Annadale Green', 'Arbutus Woods Park', 'Arden Woods', 'Alice Austen House', 'Barrett Park', 'Bayview Terrace Park', 'Blood Root Valley', 'Bloomingdale Park', 'Blue Heron Park Preserve', 'Blueberry Park', 'Bradys Park', 'Bunker Ponds Park', 'Buono Beach', 'Carlton Park', 'Clay Pit Ponds State Park Preserve (NYSP)', 'Clove Lakes Park', "Clove's Tail", 'Conference House Park', 'Corporal Thompson Park', 'Crescent Beach Park', 'Deere Park', 'Eibs Pond Park', 'Faber Pool and Park', 'Fairview Park', 'Father Macris Park', 'Forest Grove', 'Fort Hill Park', 'Fort Wadsworth (NPS - Gateway National Recreation Area)', 'South Beach–FDR Boardwalk', 'Freshkills Park', 'Gaeta Park', 'Great Kills Park (NPS - Gateway National Recreation Area)', 'Graniteville Quarry Park', 'Graniteville Swamp Park', 'Hero Park', 'High Rock Park Preserve', 'Huguenot Ponds Park', 'Hybrid Oak Woods Park', 'Ingram Woods', 'Jones Woods Park', 'Joseph Manna Park', 'King Fisher Park', 'Kingdom Pond Park', 'Last Chance Pond Park', 'LaTourette Park and Golf Course', 'Lemon Creek', 'Long Pond Park', 'General Douglas MacArthur Park', 'Maple Woods', 'Mariners Marsh Park', 'Meredith Woods', 'Midland Beach', 'Midland Field', 'Miller Field (NPS - Gateway National Recreation Area)', 'Mount Loretto Unique Area', 'New Dorp Beach', 'Northerleigh Park', 'Ocean Breeze Park', 'Old Place Creek Park', 'Olmsted-Beil House Park', "Reed's Basket Willow Swamp Park", 'Richmond Terrace Wetlands', "Sailors' Snug Harbor", 'Staten Island Botanical Garden', "New York Chinese Scholar's Garden", 'Saw Mill Creek Marsh', 'Schmul Park', 'Seaside Wildlife Nature Park', 'Siedenburg Park', 'Silver Lake', 'Sobel Court Park', 'South Beach Wetlands', 'St. George Park', 'Staten Island Greenbelt', 'Staten Island Industrial Park', 'Tappen Park', 'Tompkinsville Park', 'Tottenville Shore Park', 'Von Briesen Park', 'Walker Park', 'Wegener Park', 'Westerleigh Park', 'Westwood Park', 'Willowbrook Park', "Wolfe's Pond Park"]

    if type(s)!=str or s==None:
        return 0
    is_p=0
    for park in parks:
        ratio=fuzz.ratio(park.lower(), s.lower())
        if ratio>=85:
            is_p=1
    return is_p

In [10]:
def is_college(s):
    colleges=['Borough of Manhattan Community College', 'Bronx Community College', 'Guttman Community College', 'Hostos Community College', 'Kingsborough Community College', 'LaGuardia Community College', 'Queensborough Community College', 'Baruch College', 'Brooklyn College', 'City College of New York', 'College of Staten Island', 'Hunter College', 'John Jay College of Criminal Justice', 'Lehman College', 'Medgar Evers College', 'New York City College of Technology', 'Queens College', 'York College', 'CUNY William E. Macaulay Honors College', 'CUNY Graduate Center', 'CUNY Graduate School of Journalism', 'CUNY School of Law', 'CUNY School of Medicine', 'CUNY School of Professional Studies', 'CUNY School of Public Health', 'Fashion Institute of Technology', 'SUNY College of Optometry', 'SUNY Downstate Medical Center', 'SUNY Empire State College (The Harry Van Arsdale Jr. Center for Labor Studies)', 'SUNY Maritime College', 'The Ailey School (Alvin Ailey American Dance Crew).', 'Albert Einstein College of Medicine', 'American Academy of Dramatic Arts', 'American Academy McAllister Institute', 'American Musical and Dramatic Academy', 'ASA College', 'Bank Street College of Education', 'Bard College', 'Bard Graduate Center', 'Globalization and International Affairs Program', 'Barnard College', 'Berkeley College', 'Bethel Seminary of the East', 'Boricua College', 'Bramson ORT College', 'Briarcliffe College - The Queens Center', 'Brooklyn Law School', "Christie's Education", 'College of Mount Saint Vincent', 'College of New Rochelle (School of New Resources)', 'Columbia University', 'Teachers College', 'Union Theological Seminary', 'Columbia University School of General Studies', 'Cooper Union', 'Cornell University', 'Cornell NYC Tech', 'Weill Cornell Graduate School of Medical Sciences', 'Weill Cornell Medical College', 'DeVry University', 'European School of Economics', 'Fordham University', 'Frank G. Zarb School of Business at Hofstra University[1]', 'General Theological Seminary', 'Gerstner Sloan Kettering Graduate School of Biomedical Science', 'Globe Institute of Technology', 'Hebrew Union College', 'Helene Fuld College of Nursing', 'Icahn School of Medicine at Mount Sinai', 'Jewish Theological Seminary of America', 'The Juilliard School', 'Keller Graduate School of Management', "The King's College", 'Laboratory Institute of Merchandising', 'Long Island Business Institute - Flushing', 'Long Island College Hospital School of Nursing', 'Long Island University', 'Mandl College of Allied Health', 'Manhattan College', 'Manhattan School of Music', 'Marymount Manhattan College', 'Mercy College', 'Metropolitan College of New York', 'Monroe College', 'The New School', 'Eugene Lang College The New School for Liberal Arts', 'Mannes College The New School for Music', 'Milano School of Management, Policy, and Environment', 'The New School for Drama', 'The New School for General Studies', 'The New School for Jazz and Contemporary Music', 'The New School for Social Research', 'Parsons The New School for Design', 'New York Academy of Art', 'New York Career Institute', 'New York Conservatory for Dramatic Arts', 'New York Film Academy', 'New York Graduate School of Psychoanalysis', 'New York Institute of Technology', 'New York Law School', 'New York School of Interior Design', 'New York School of Urban Ministry', 'New York Theological Seminary', 'New York University', 'Gallatin School of Individualized Study', 'New York University Polytechnic School of Engineering', 'New York University School of Law', 'New York University School of Medicine', 'Tisch School of Arts', 'New York University of Architecture', 'Nyack College (School of Music)', 'Pace University', 'Pacific College of Oriental Medicine', 'Phillips Beth Israel School of Nursing', 'Plaza College', 'Pratt Institute', 'Rabbi Isaac Elchanon Theological Seminary', 'Richard Gilder Graduate School - American Museum of Natural History', 'Rockefeller University', 'School of American Ballet', 'School of Visual Arts', 'St. Francis College', "St. John's University", 'New Brunswick Theological Seminary', "St. Joseph's College", "Sotheby's Institute of Art", 'Studio Maestro', 'Swedish Institute of Massage Therapy', 'Technical Career Institute College of Technology', 'Touro College', 'Tri-State College of Acupuncture', 'Vaughn College of Aeronautics & Technology', 'Wagner College', 'Wood Tobe-Coburn School', 'Yeshiva University', 'Benjamin N. Cardozo School of Law']

    if type(s)!=str or s==None:
        return 0
    is_uni=0
    for uni in colleges:
        ratio=fuzz.ratio(uni.lower(), s.lower())
        if ratio>=85:
            is_uni=1
    return is_uni

In [11]:
import re
def process_website(word):
    if word==None or type(word)!=str:
        return 0
    is_website=0
    ls = re.split('\.|\/',word.lower())
    lst = ['gov', 'com', 'org', 'info', 'http', 'https', 'net','edu','www','me','info',
          'ai', 'nyc']
    if any(item in ls for item in lst):
        is_website=1
    else:
        is_website=0
    return is_website

In [12]:
def is_agency(s):
    agencies=['Actuary, NYC Office of the (NYCOA)', 'Administrative Justice Coordinator, NYC Office of (AJC)', 'Administrative Tax Appeals, Office of', 'Administrative Trials and Hearings, Office of (OATH)', 'Aging, Department for the (DFTA)', "Appointments, Mayor's Office of (MOA)", 'Brooklyn Public Library (BPL)', 'Buildings, Department of (DOB)', 'Business Integrity Commission (BIC)', 'Campaign Finance Board (CFB)', 'Center for Innovation through Data Intelligence (CIDI)', 'Charter Revision Commission', 'Chief Medical Examiner, NYC Office of (OCME)', "Children's Services, Administration for (ACS)", 'City Clerk, Office of the (CLERK)', 'City Council, New York', 'City Planning, Department of (DCP)', 'City University of New York (CUNY)', 'Citywide Administrative Services, Department of (DCAS)', 'Citywide Event Coordination and Management, Office of (CECM)', 'Civic Engagement Commission (CEC)', 'Civil Service Commission (CSC)', 'Civilian Complaint Review Board (CCRB)', 'Climate Policy & Programs', 'Commission on Gender Equity (CGE)', 'Commission to Combat Police Corruption (CCPC)', 'Community Affairs Unit (CAU)', 'Community Boards (CB)', 'Comptroller (COMP)', 'Conflicts of Interest Board (COIB)', 'Consumer Affairs, Department of (DCA)', 'Consumer and Worker Protection, Department of (DCWP)', "Contract Services, Mayor's Office of (MOCS)", 'Correction, Board of (BOC)', 'Correction, Department of (DOC)', "Criminal Justice, Mayor's Office of", 'Cultural Affairs, Department of (DCLA)', "Data Analytics, Mayor's Office of (MODA)", 'Design and Construction, Department of (DDC)', 'District Attorney - Bronx County', 'District Attorney - Kings County (Brooklyn)', 'District Attorney - New York County (Manhattan)', 'District Attorney - Queens County', 'District Attorney - Richmond County (Staten Island)', 'Education, Department of (DOE)', 'Elections, Board of (BOE)', 'Emergency Management, NYC', 'Environmental Coordination, Mayor’s Office of (MOEC)', 'Environmental Protection, Department of (DEP)', 'Equal Employment Practices Commission (EEPC)', 'Finance, Department of (DOF)', 'Fire Department, New York City (FDNY)', 'Fiscal Year 2005 Securitization Corporation', 'Food Policy Director, Office of the', 'GreeNYC (GNYC)', 'Health and Mental Hygiene, Department of (DOHMH)', 'Homeless Services, Department of (DHS)', 'Housing Authority, New York City (NYCHA)', 'Housing Preservation and Development, Department of (HPD)', 'Housing Recovery Operations (HRO)', 'Hudson Yards Infrastructure Corporation', 'Human Resources Administration (HRA)', 'Human Rights, City Commission on (CCHR)', "Immigrant Affairs, Mayor's Office of (MOIA)", 'Independent Budget Office, NYC (IBO)', "Information Privacy, Mayor's Office of (MOIP)", 'Information Technology and Telecommunications, Department of (DOITT)', 'Inspector General NYPD, Office of the', "Intergovernmental Affairs, Mayor's Office of (MOIGA)", 'Investigation, Department of (DOI)', "Judiciary, Mayor's Advisory Committee on the (MACJ)", 'Labor Relations, NYC Office of (OLR)', 'Landmarks Preservation Commission (LPC)', 'Law Department (LAW)', 'Library, Brooklyn Public (BPL)', 'Library, New York Public (NYPL)', 'Library, Queens Public (QL)', 'Loft Board (LOFT)', 'Management and Budget, Office of (OMB)', "Mayor's Committee on City Marshals (MCCM)", "Mayor's Fund to Advance NYC (Mayor's Fund)", "Mayor's Office (OM)", "Mayor's Office for Economic Opportunity", "Mayor's Office for International Affairs (IA)", "Mayor's Office for People with Disabilities (MOPD)", "Mayor's Office of Environmental Remediation (OER)", "Mayor's Office of Special Projects & Community Events (MOSPCE)", "Mayor's Office of the Chief Technology Officer", 'Mayor’s Office of Minority and Women-Owned Business Enterprises (OMWBE)', 'Mayor’s Office of Strategic Partnerships (OSP)', 'Mayor’s Office to End Domestic and Gender-Based Violence (ENDGBV)', "Media and Entertainment, Mayor's Office of (MOME)", 'Media, NYC', 'NYC & Company (NYCGO)', "NYC Children's Cabinet", 'NYC Cyber Command', 'NYC Economic Development Corporation (NYCEDC)', "NYC Employees' Retirement System (NYCERS)", 'NYC Health + Hospitals', 'NYC Service (SERVICE)', 'NYC Young Men’s Initiative', 'New York City Transitional Finance Authority (TFA)', 'New York Public Library (NYPL)', 'Office of Recovery & Resiliency', 'Office of ThriveNYC', 'Office of the Census for NYC', "Operations, Mayor's Office of (OPS)", 'Parks and Recreation, Department of (DPR)', 'Payroll Administration, Office of (OPA)', 'Police Department (NYPD)', 'Police Pension Fund (PPF)', 'Probation, Department of (DOP)', 'Procurement Policy Board (PPB)', 'Property Tax Reform, Advisory Commission on', 'Public Administrator - Bronx County (BCPA)', 'Public Administrator - Kings County (KCPA)', 'Public Administrator - New York County (NYCountyPA)', 'Public Administrator - Queens County (QPA)', 'Public Administrator - Richmond County (RCPA)', 'Public Advocate (PUB ADV)', 'Public Design Commission', 'Queens Public Library (QPL)', 'Records and Information Services, Department of (DORIS)', 'Rent Guidelines Board (RGB)', 'Sales Tax Asset Receivable Corporation (STAR)', 'Sanitation, Department of (DSNY)', 'School Construction Authority (SCA)', 'Small Business Services (SBS)', 'Social Services, Department of (DSS)', 'Special Commissioner of Investigation for the New York City School District', 'Special Enforcement, Mayor’s Office of (OSE)', 'Special Narcotics Prosecutor, NYC Office of the (SNP)', 'Standards and Appeals, Board of (BSA)', "Sustainability, Mayor's Office Of", 'TSASC, Inc.', 'Tax Appeals Tribunal, New York City (TAT)', 'Tax Commission, New York City (TC)', 'Taxi and Limousine Commission (TLC)', "Teachers' Retirement System of the City of New York", 'Transportation, Department of (DOT)', "Veterans' Services, Department of (DVS)", 'Water Board (NYWB)', 'Water Finance Authority, NYC Municipal (NYW)', "Workforce Development, Mayor's Office of", 'Youth and Community Development, Department of (DYCD)']

    if type(s)!=str or s==None:
        return 0
    is_gov_agency=0
    for agency in agencies:
        ratio=fuzz.partial_ratio(agency.lower(), s.lower())
        if ratio>=85:
            is_gov_agency=1
    return is_gov_agency

In [13]:
def is_major(s):
    majors=['Accounting', 'Actuarial Science', 'Advertising', 'Aerospace Engineering', 'African Languages, Literatures, and Linguistics', 'African Studies', 'African-American Studies', 'Agricultural Business and Management', 'Agricultural Economics', 'Agricultural Education', 'Agricultural Journalism', 'Agricultural Mechanization', 'Agricultural Technology Management', 'Agricultural/Biological Engineering and Bioengineering', 'Agriculture', 'Agronomy and Crop Science', 'Air Traffic Control', 'American History', 'American Literature', 'American Sign Language', 'American Studies', 'Anatomy', 'Ancient Studies', 'Animal Behavior and Ethology', 'Animal Science', 'Animation and Special Effects', 'Anthropology', 'Applied Mathematics', 'Aquaculture', 'Aquatic Biology', 'Arabic', 'Archeology', 'Architectural Engineering', 'Architectural History', 'Architecture', 'Art', 'Art Education', 'Art History', 'Art Therapy', 'Artificial Intelligence and Robotics', 'Asian-American Studies', 'Astronomy', 'Astrophysics', 'Athletic Training', 'Atmospheric Science', 'Automotive Engineering', 'Aviation', 'Bakery Science', 'Biblical Studies', 'Biochemistry', 'Bioethics', 'Biology', 'Biomedical Engineering', 'Biomedical Science', 'Biopsychology', 'Biotechnology', 'Botany/Plant Biology', 'Business Administration/Management', 'Business Communications', 'Business Education', 'Canadian Studies', 'Caribbean Studies', 'Cell Biology', 'Ceramic Engineering', 'Ceramics', 'Chemical Engineering', 'Chemical Physics', 'Chemistry', 'Child Care', 'Child Development', 'Chinese', 'Chiropractic', 'Church Music', 'Cinematography and Film/Video Production', 'Circulation Technology', 'Civil Engineering', 'Classics', 'Clinical Psychology', 'Cognitive Psychology', 'Communication Disorders', 'Communications Studies/Speech Communication and Rhetoric', 'Comparative Literature', 'Computer and Information Science', 'Computer Engineering', 'Computer Graphics', 'Computer Systems Analysis', 'Construction Management', 'Counseling', 'Crafts', 'Creative Writing', 'Criminal Science', 'Criminology', 'Culinary Arts', 'Dance', 'Data Processing', 'Dental Hygiene', 'Developmental Psychology', 'Diagnostic Medical Sonography', 'Dietetics', 'Digital Communications and Media/Multimedia', 'Drawing', 'Early Childhood Education', 'East Asian Studies', 'East European Studies', 'Ecology', 'Economics', 'Education', 'Education Administration', 'Education of the Deaf', 'Educational Psychology', 'Electrical Engineering', 'Elementary Education', 'Engineering Mechanics', 'Engineering Physics', 'English', 'English Composition', 'English Literature', 'Entomology', 'Entrepreneurship', 'Environmental Design/Architecture', 'Environmental Science', 'Environmental/Environmental Health Engineering', 'Epidemiology', 'Equine Studies', 'Ethnic Studies', 'European History', 'Experimental Pathology', 'Experimental Psychology', 'Fashion Design', 'Fashion Merchandising', 'Feed Science', 'Fiber, Textiles, and Weaving Arts', 'Film', 'Finance', 'Floriculture', 'Food Science', 'Forensic Science', 'Forestry', 'French', 'Furniture Design', 'Game Design', 'Gay and Lesbian Studies', 'Genetics', 'Geography', 'Geological Engineering', 'Geology', 'Geophysics', 'German', 'Gerontology', 'Government', 'Graphic Design', 'Health Administration', 'Hebrew', 'Hispanic-American, Puerto Rican, and Chicano Studies', 'Historic Preservation', 'History', 'Home Economics', 'Horticulture', 'Hospitality', 'Human Development', 'Human Resources Management', 'Illustration', 'Industrial Design', 'Industrial Engineering', 'Industrial Management', 'Industrial Psychology', 'Information Technology', 'Interior Architecture', 'Interior Design', 'International Agriculture', 'International Business', 'International Relations', 'International Studies', 'Islamic Studies', 'Italian', 'Japanese', 'Jazz Studies', 'Jewelry and Metalsmithing', 'Jewish Studies', 'Journalism', 'Kinesiology', 'Korean', 'Land Use Planning and Management', 'Landscape Architecture', 'Landscape Horticulture', 'Latin American Studies', 'Library Science', 'Linguistics', 'Logistics Management', 'Management Information Systems', 'Managerial Economics', 'Marine Biology', 'Marine Science', 'Marketing', 'Mass Communication', 'Massage Therapy', 'Materials Science', 'Mathematics', 'Mechanical Engineering', 'Medical Technology', 'Medieval and Renaissance Studies', 'Mental Health Services', 'Merchandising and Buying Operations', 'Metallurgical Engineering', 'Microbiology', 'Middle Eastern Studies', 'Military Science', 'Mineral Engineering', 'Missions', 'Modern Greek', 'Molecular Biology', 'Molecular Genetics', 'Mortuary Science', 'Museum Studies', 'Music', 'Music Education', 'Music History', 'Music Management', 'Music Therapy', 'Musical Theater', 'Native American Studies', 'Natural Resources Conservation', 'Naval Architecture', 'Neurobiology', 'Neuroscience', 'Nuclear Engineering', 'Nursing', 'Nutrition', 'Occupational Therapy', 'Ocean Engineering', 'Oceanography', 'Operations Management', 'Organizational Behavior Studies', 'Painting', 'Paleontology', 'Pastoral Studies', 'Peace Studies', 'Petroleum Engineering', 'Pharmacology', 'Pharmacy', 'Philosophy', 'Photography', 'Photojournalism', 'Physical Education', 'Physical Therapy', 'Physician Assistant', 'Physics', 'Physiological Psychology', 'Piano', 'Planetary Science', 'Plant Pathology', 'Playwriting and Screenwriting', 'Political Communication', 'Political Science', 'Portuguese', 'Pre-Dentistry', 'Pre-Law', 'Pre-Medicine', 'Pre-Optometry', 'Pre-Seminary', 'Pre-Veterinary Medicine', 'Printmaking', 'Psychology', 'Public Administration', 'Public Health', 'Public Policy Analysis', 'Public Relations', 'Radio and Television', 'Radiologic Technology', 'Range Science and Management', 'Real Estate', 'Recording Arts Technology', 'Recreation Management', 'Rehabilitation Services', 'Religious Studies', 'Respiratory Therapy', 'Risk Management', 'Rural Sociology', 'Russian', 'Scandinavian Studies', 'Sculpture', 'Slavic Languages and Literatures', 'Social Psychology', 'Social Work', 'Sociology', 'Soil Science', 'Sound Engineering', 'South Asian Studies', 'Southeast Asia Studies', 'Spanish', 'Special Education', 'Speech Pathology', 'Sport and Leisure Studies', 'Sports Management', 'Statistics', 'Surveying', 'Sustainable Resource Management', 'Teacher Education', 'Teaching English as a Second Language', 'Technical Writing', 'Technology Education', 'Textile Engineering', 'Theatre', 'Theology', 'Tourism', 'Toxicology', 'Turfgrass Science', 'Urban Planning', 'Urban Studies', 'Visual Communication', 'Voice', 'Web Design', 'Webmaster and Web Management', 'Welding Engineering', 'Wildlife Management', "Women's Studies", 'Youth Ministries', 'Zoology']

    if type(s)!=str or s==None:
        return 0
    is_area=0
    
    for area in majors:
        ratio=fuzz.ratio(area.lower(), s.lower())
        if ratio>=85:
            is_area=1
    return is_area

In [14]:
def school_subject(s):
    school_subjects=['Math 7 (Grade Level Class for 7th Graders)', 'Pre-Algebra (Above Grade Level, but most students take this)', 'Algebra 1 (High School Course; most 8th graders take this, some really advanced 7th graders take this)', 'Math 8 (Grade Level Class for 8th Graders)', 'Geometry (High School Course for very advanced 8th graders)', 'Math Workshop (Extra Class to assist below grade level math students)', 'English:', 'English 7 (Grade level English class for 7th Graders)', 'Advanced English 7 (Advanced English class for 7th Graders)', 'English 8 (Grade Level English class for 8th Graders)', 'Advanced English 8 (Advance English class for 8th Graders)', 'Reading Workshop (Extra Class to assist below grade level English students)', 'Science:', 'Science 7 (Grade level Science class for 7th graders)', 'Advanced Science 7 (Advanced Science class for 7th graders)', 'Science 8 (Grade Level Science class for 8th graders)', 'Advanced Science 8 (Advance Science class for 8th graders)', 'Social Studies:', 'Social Studies 7 (Social Studies Class for 7th graders)', 'Social Studies 8 (Grade Level Social Studies Class for 8th Graders)', 'Advanced Social Studies 8 (Advanced Social Studies class for 8th Graders)', 'Electives (* means required to leave middle school):', 'Spanish', 'Spanish 1', 'Spanish 2', 'Spanish for Native Speakers', 'French', 'French 1', 'French 2', 'Introduction to World Languages', 'Art', 'Art Exploration', 'Art 2D', 'Art 3D', 'Humanities', 'STEM', 'GTT: Design and Modeling', 'GTT: Robotics and Automation', 'Projects in Industrial Technology (Woodshop)', 'Business', 'Career Explorations and Money Management', 'Computer Dimensions 1', 'Computer Dimensions 2', 'Family Science', 'Introduction to Family and Consumer Sciences', 'Exploring Sewing Technology', 'Exploring Foods and Nutrition', 'Performing Arts', 'Theater', 'Performance 1', 'Performance 2', 'Band', 'Beginning Band', 'Band 1', 'Band 2', 'Orchestra/Strings', 'Beginning Orchestra', 'Orchestra 1', 'Orchestra 2', 'Choir', 'Vocal Music', 'Chorus 1', 'Chorus 2', 'Creative Writing', 'SEEK (Gifted Class)', 'Physical Education*']

    if type(s)!=str or s==None :
        return 0
    is_sub=0
    
    for sub in school_subjects:
        ratio=fuzz.ratio(sub.lower(), s.lower())
        if ratio>=85:
            is_sub=1
    return is_sub

In [15]:
def is_school(s):
    schools=['Academy for Language and Technology', 
             'Academy for Scholarship and Entrepreneurship: A College Board School', 
             'Academy of Mount Saint Ursula', 'Adlai E. Stevenson Educational Campus', 
             'Adlai E. Stevenson High School (closed 2009)', 'Bronx Guild', 
             'Gateway School for Environmental Research and Technology', 
             'Millennium Art Academy', 'Pablo Neruda Academy for Architecture and World Studies', 
             'School for Community Research and Learning', '(split)', 
             'Alfred E. Smith Career and Technical Education High School', 'Collocated schools:', 
             'Bronx Design and Construction Academy', 'Bronx Haven High School', 'X600', 'All Hallows High School', 
             'Aquinas High School', 'Astor Collegiate Academy', '(Christopher Columbus campus)', 
             'Banana Kelly High School', '(Collocated with Holcombe L. Rucker School of Community Research)', 
             'Belmont Preparatory High School', '(Theodore Roosevelt campus)', 'Bronx Academy of Health Careers', 
             '(Evander Childs campus)', 'Bronx Academy High School', 'Bronx Academy of Letters', 
             'Bronx Aerospace High School', '(Evander Childs campus)', 'Bronx Center for Science and Mathematics', 'Bronx Charter School for Excellence', 'Bronx Coalition Community High School', '(James Monroe Campus)', 'Bronx Collegiate Academy', '(formerly Bronx Expeditionary Learning High School)', '(William H. Taft campus)', 'Bronx Community High School', 'Bronx Design and Construction Academy', '(Alfred E. Smith Educational Campus)', 'Bronx Engineering and Technology Academy', '(John F. Kennedy campus)', 'Bronx Envision Academy', '(Collocated at Explorations Academy)', 'Bronx Guild', '(Adlai E. Stevenson Educational Campus)', 'Bronx Haven High School', '(Alfred E.Smith Campus)', 'Bronx High School for Law and Community Service', 'Theodore Roosevelt campus', 'Bronx High School for Medical Science', '(William H. Taft campus)', 'Bronx High School for the Visual Arts', 'Bronx High School for Writing and Communication Arts', '(Evander Childs campus)', 'Bronx High School of Business', '(William H. Taft campus)', 'Bronx High School of Science', 'Bronx International High School', '(Morris Campus)', 'Bronx Lab School', '(Evander Childs campus)', 'Bronx Latin', 'Bronx Leadership Academy High School', 'Bronx Leadership Academy II High School', '(Concourse Village campus)', 'Bronx Preparatory Charter School', 'Bronx Regional High School', 'Bronx School for Law, Government and Justice', 'Bronx School of Law and Finance', '(John F. Kennedy campus)', 'Bronx Studio School for Writers and Artists', 'Bronx Theatre High School', '(John F. Kennedy campus)', 'Bronxdale High School', '(Christopher Columbus campus)', 'The Bronxwood Preparatory Academy', 'Cardinal Hayes High School', 'Cardinal Spellman High School', 'The Celia Cruz Bronx High School of Music', '(Walton Educational Campus)', 'Christopher Columbus High School', 'Collocated specialty schools:', 'Astor Collegiate Academy', 'Bronxdale High School', 'Collegiate Institute for Math and Science', 'Global Enterprise High School', 'Pelham Preparatory Academy', 'X415', 'The Cinema School', '(James Monroe Campus)', 'Collegiate Institute for Math and Science', '(Christopher Columbus campus)', 'Community School for Social Justice', 'Concourse Village campus', 'See:', 'Bronx Leadership Academy II', 'New Explorers High School', 'Urban Assembly School for Careers in Sports', '(multiple)', 'Crotona Academy High School', 'DeWitt Clinton High School', 'Discovery High School', '(Walton Educational Campus)', 'DreamYard Preparatory School', '(William H. Taft campus)', 'Eagle Academy for Young Men', 'East Bronx Academy for the Future', 'Evander Childs Educational Campus', 'See:', 'Bronx Academy of Health Careers', 'Bronx Aerospace High School', 'Bronx High School for Writing and Communication Arts', 'Bronx Lab School', 'High School for Contemporary Arts', 'High School of Computers and Technology', '(split)', 'Eximius College Preparatory Academy: A College Board School', 'Explorations Academy', 'Collocated school: Bronx Envision Academy', 'Fannie Lou Hamer Freedom High School', 'The Felisa Rincon de Gautier Institute for Law and Public Policy', 'The Fieldston School', '(Ethical Culture Fieldston School)', 'Fordham High School for the Arts', '(Theodore Roosevelt campus)', 'Fordham Leadership Academy for Business and Technology', '(Theodore Roosevelt campus)', 'Fordham Preparatory School', 'Foreign Language Academy of Global Studies (FLAGS)', 'Frederick Douglass Academy III Secondary School', 'Gateway School for Environmental Research and Technology', '(Adlai E. Stevenson Educational Campus)', 'Global Enterprise High School', '(Christopher Columbus campus)', 'Grace Dodge Career and Technical Education High School', 'Harry S Truman High School', 'Health Opportunities High School', 'Herbert H. Lehman High School', 'Collocated school:', 'Renaissance High School for Musical Theater & Technology', 'X405', 'High School for Contemporary Arts', '(Evander Childs campus)', 'High School for Language and Innovation', '(Christopher Columbus Educational Campus)', 'High School for Teaching and the Professions', '(Walton Educational Campus)', 'High School for Violin and Dance', '(Morris Campus)', 'High School of American Studies at Lehman College', 'High School of Computers and Technology', '(Evander Childs campus)', 'High School of World Cultures', '(James Monroe Campus)', 'Holcombe L. Rucker School of Community Research', '(Collocated with Banana Kelly High School)', 'Horace Mann School', 'Hostos-Lincoln Academy of Science', 'In-Tech Academy', '(Information and Network Technology Academy)', 'International Community High School', 'International School for Liberal Arts', '(Walton Educational Campus)', 'James Monroe Educational Campus', 'See:', 'James Monroe High School (closed)', 'Bronx Coalition Community High School', 'The Cinema School', 'High School of World Cultures', 'The Metropolitan Soundview High School', 'Monroe Academy for Business/Law', 'Monroe Academy for Visual Arts & Design', 'Pan American International High School at Monroe', '(split)', 'Jane Addams High School for Academic Careers', 'Jill Chaifetz Transfer High School', 'John F. Kennedy High School', 'Collocated schools:', 'Bronx Engineering and Technology Academy', 'Bronx School of Law and Finance', 'Bronx Theatre High School', 'Marble Hill High School for International Studies', 'X475', 'Jonathan Levin High School for Media and Communications', '(William H. Taft campus)', 'Kingsbridge International High School', '(Walton Educational Campus)', 'Knowledge and Power Preparatory Academy International High School (KAPPA)', '(Theodore Roosevelt campus)', 'Leadership Institute', 'Marble Hill High School for International Studies', '(John F. Kennedy campus)', 'The Marie Curie School for Medicine, Nursing, and Health Professions', 'The Metropolitan High School', 'The Metropolitan Soundview High School', '(James Monroe Educational Campus)', 'Millennium Art Academy', '(Adlai E. Stevenson Educational Campus)', 'Monroe Academy for Business/Law', '(James Monroe Campus)', 'Monroe Academy for Visual Arts & Design', '(James Monroe Campus)', 'Monsignor Scanlan High School', 'Morris Academy for Collaborative Studies', '(Morris Campus)', 'Morris Educational Campus', 'See:', 'Morris High School (closed)', 'Bronx International High School', 'High School for Violin and Dance', 'Morris Academy for Collaborative Studies', 'School for Excellence', '(split)', 'Mott Hall Bronx High School', 'Mott Haven Village Preparatory High School', '(South Bronx campus)', 'Mount Saint Michael Academy', 'New Day Academy', 'New Explorers High School', '(Concourse Village campus)', 'New World High School', 'Pablo Neruda Academy for Architecture and World Studies', '(Adlai E. Stevenson Educational Campus)', 'Pan American International High School at Monroe', '(James Monroe Campus', 'Latino immigrants only', 'Peace and Diversity Academy', 'Pelham Preparatory Academy', '(Christopher Columbus campus)', 'Performance Conservatory High School', '(Bronx High School for Performance and Stagecraft)', 'Preston High School', 'Providing Urban Learners Success in Education (PULSE) High School', 'Renaissance High School for Musical Theater & Technology', '(Herbert H. Lehman Educational Campus)', 'Riverdale Country School', 'Riverdale Kingsbridge Academy', 'St. Barnabas High School', 'St. Catharine Academy', 'St. Pius V High School', 'St. Raymond Academy for Girls', 'St. Raymond High School for Boys', 'Salanter Akiba of Riverdale (SAR) High School', 'Samuel Gompers Career and Technical Education High School', 'School for Community Research and Learning', '(Adlai E. Stevenson Educational Campus)', 'School for Excellence', '(Morris Campus)', 'South Bronx Educational Campus', 'See:', 'South Bronx High School (closed)', 'Mott Haven Village Preparatory High School', 'University Heights Secondary School', '(split)', 'South Bronx Preparatory: A College Board School', 'Theatre Arts Production Company School', 'Theodore Roosevelt Educational Campus', 'See:', 'Theodore Roosevelt High School (closed)', 'Belmont Preparatory High School', 'Bronx High School for Law and Community Service', 'Fordham High School for the Arts', 'Fordham Leadership Academy for Business and Technology', 'Knowledge and Power Preparatory Academy International High School (KAPPA)', 'West Bronx Academy for the Future', '(split)', 'University Heights Secondary School', '(South Bronx campus)', 'The Urban Assembly Academy for History and Citizenship for Young Men (closed 2014)', '(William H. Taft campus)', 'The Urban Assembly School for Applied Math and Science', 'Careers in Sports High School', '(Concourse Village campus)', 'Validus Preparatory Academy: An Expeditionary Learning School', 'Walton Educational Campus', 'See:', 'Walton High School (closed)', 'Celia Cruz Bronx High School of Music', 'Discovery High School', 'High School for Teaching and the Professions', 'International School for Liberal Arts', 'Kingsbridge International High School', '(split)', 'West Bronx Academy for the Future', '(Theodore Roosevelt campus)', 'William Howard Taft Educational Campus', 'See:', 'William H. Taft High School (closed)', 'Bronx Collegiate Academy', 'Bronx High School for Medical Science', 'Bronx High School of Business', 'DreamYard Preparatory School', 'Jonathan Levin High School for Media and Communications', 'The Urban Assembly Academy for History and Citizenship for Young Men', '(split)', 'Wings Academy', "Women's Academy of Excellence", 'Yeshiva of Telshe Alumni', 'Brooklyn', 'This section: viewtalkedit', 'School', 'NYC DOE', 'Abraham Lincoln High School', 'Academy for College Preparation and Career Exploration:A College Board School', '(Erasmus Hall Educational Campus)', 'Academy for Conservation and the Environment', '(South Shore Educational Campus)', 'Academy for Environmental Leadership', '(Bushwick Educational Campus)', 'Academy for Young Writers', 'Academy of Hospitality and Tourism', '(Erasmus Hall Educational Campus)', 'Academy of Innovative Technology', 
             '(Franklin K. Lane Educational Campus)', 'Academy of Urban Planning', '(Bushwick Educational Campus)', 'ACORN Community High School', 'All City Leadership Secondary School', 'Al-Madinah School', 'Al-Noor School', 'Aspirations Diploma Plus High School', 'Automotive High School', 'Bedford Academy High School', 'Bedford Stuyvesant Preparatory High School', 'Benjamin Banneker Academy', 'Berkeley Carroll School', 'Beth Jacob High School', 'Beth Rivkah', 'Big Apple Academy', 'Bishop Ford Central Catholic High School', 'Bishop Kearney High School', 'Bishop Loughlin Memorial High School', 'B’nos Leah Prospect Park Yeshiva School', 'B’nos Yisroel High School for Girls', 'Boys and Girls High School', 'Brooklyn Academy of Science and the Environment (BASE)', '(Prospect Heights Educational Campus)', 'Brooklyn Academy High School', 'Brooklyn Bridge Academy', '(South Shore Educational Campus)', 'Brooklyn College Academy', 'Brooklyn Collegiate: A College Board School', 'Brooklyn Community High School of Communication, Arts and Media', 'Brooklyn Democracy Academy', 'Brooklyn Generation School', '(South Shore Educational Campus)', 'Brooklyn High School for Leadership and Community Service', 'Brooklyn High School of the Arts', 'Brooklyn High School for Law and Technology', '(Formerly ACORN High School for Social Justice)', 'Brooklyn International High School', '(Waters Edge Educational Complex)', 'Brooklyn Lab School', '(Franklin K. Lane Educational Campus)', 'The Brooklyn Latin School', 'Brooklyn Preparatory High School', '(Harry Van Arsdale Educational Campus)', 'Brooklyn School for Global Studies', 'The Brooklyn School for Math and Research', '(Bushwick Educational Campus)', 'Brooklyn School for Music & Theatre', '(Prospect Heights Educational Campus)', 'Brooklyn Secondary School for Collaborative Studies', 'Brooklyn Studio Secondary School', 'Brooklyn Technical High School', 'Brooklyn Theatre Arts High School', '(South Shore Educational Campus)', 'Brownsville Academy High School', 'Bushwick Community High School', 'Bushwick Educational Campus', 'See:', 'Bushwick High School (closed 2006)', 'Academy for Environmental Leadership', 'Academy of Urban Planning', 'The Brooklyn School for Math and Research', 'Bushwick School for Social Justice', '(split)', 'Bushwick Leaders High School for Academic Excellence', 'Bushwick School for Social Justice', '(Bushwick Educational Campus)', 'Canarsie High School', 'Catherine McAuley High School', 'City Polytechnic High School of Engineering, Architecture, and Technology', 'Clara Barton High School', 'Cobble Hill School of American Studies', 'Connie Lekas School', 'Cypress Hills Collegiate Preparatory School', '(Franklin K. Lane Educational Campus)', 'Dr. Susan S. McKinney Secondary School of the Arts', 'East New York Family Academy', 'EBC/ENY High School for Public Safety & Law', 'EBC High School for Public Service–Bushwick', 'Edward R. Murrow High School', 'El Puente Academy for Peace and Justice', 'Enterprise, Business and Technology High School', 'Expeditionary Learning School for Community Leaders', 'Erasmus Hall Educational Campus', 'See:', 'Erasmus Hall High School (closed 1994)', 'Academy for College Preparation and Career Exploration:A College Board School', 'Academy of Hospitality and Tourism', 'High School for Service & Learning at Erasmus', 'High School for Youth and Community Development at Erasmus', 'Science, Technology and Research Early College High School at Erasmus', '(split)', 'FDNY High School for Fire and Life Safety', '(Thomas Jefferson Educational Campus)', 'Fontbonne Hall Academy', 'Fort Hamilton High School', 'Foundations Academy', 'Franklin Delano Roosevelt High School', 'Franklin K. Lane Educational Campus', 'See:', 'Franklin K. Lane High School (closing 2012)', 'Academy of Innovative Technology', 'Brooklyn Lab School', 'Cypress Hills Collegiate Preparatory School', 'Multicultural High School', 'K420', 'Frederick Douglass Academy IV Secondary School', 'Frederick Douglass Academy VII High School', 'Freedom Academy High School', 'George Wingate Educational Campus', 'See:', 'George W. Wingate High School (closed 2000)', 'High School for Public Service: Heroes of Tomorrow', 'International Arts Business School', 'School for Democracy and Leadership', 'The School for Human Rights', '(split)', 'Gerer Mesivta Bais Yisroel School', 'George Westinghouse Career and Technical Education High School', 'Gotham Professional Arts Academy', 'Green School: An Academy for Environmental Careers', 'Harry Van Arsdale Educational Campus', 'See:', 'Harry Van Arsdale High School (closed 2007)', 'Brooklyn Preparatory High School', 'Williamsburg High School for Architecture and Design', 'Williamsburg Preparatory School', '(split)', 'High School for Civil Rights', '(Thomas Jefferson Educational Campus)', 'The High School for Global Citizenship', '(Prospect Heights Educational Campus)', 'High School for Public Service: Heroes of Tomorrow', '(George W. Wingate Educational Campus)', 'High School for Service & Learning at Erasmus', '(Erasmus Hall Educational Campus)', 'High School for Youth and Community Development at Erasmus', '(Erasmus Hall Educational Campus)', 'High School of Sports Management', 'High School of Telecommunication Arts and Technology', 'International Arts Business School', '(George W. Wingate Educational Campus)', 'International High School at Lafayette', 'International High School at Prospect Heights', 'Prospect Heights Educational Campus', 'It Takes a Village Academy', '(Samuel J. Tilden Educational Campus)', 'James Madison High School', 'John Dewey High School', 'John Jay Educational Campus', 'See:', 'John Jay High School (closed 2003)', 'Millennium Brooklyn High School', 'Park Slope Collegiate (formerly Secondary School for Research)', 'Secondary School for Journalism', 'Secondary School for Law', '(split)', 'Juan Morel Campos Secondary School', 'Kingsborough Early College Secondary School', 'Kurt Hahn Expeditionary Learning School', '(Samuel J. Tilden Educational Campus)', 'Lafayette High School', 'Leon M. Goldstein High School for the Sciences', 'Liberation Diploma Plus', 'Life Academy High School for Film and Music', 'Lyons Community School', 'Magen David Yeshivah High School', 'Medgar Evers College Preparatory School', 'Merkaz Bnos High School', "Mesivta M'kor Chaim School", 'Mesivta Nachlas Yakov School', 'Mesivta of Seagate School', 'Mesivta Rabbi Chaim Berlin High School', 'Mesivta/Yeshiva Gedola Manhattan Beach School', 'Metropolitan Corporate Academy', 'Metropolitan Diploma Plus High School', 'Midwood High School', 'Mikdash Melech Mechina School', 'Millennium Brooklyn High School', '(John Jay Educational Campus)', 'Mirrer Yeshiva High School', 'Multicultural High School', '(Franklin K. Lane Educational Campus)', 'Nazareth Regional High School', 'New Utrecht High School', 'New York Harbor School', 'Olympus Academy', 'Pacific High School', 'The Packer Collegiate Institute', 'Park Slope Collegiate', '(John Jay Educational Campus)', '(Formerly Secondary School for Research)', 'Pathways in Technology Early College High School', '(Collocated at Paul Robeson High School)', 'Paul Robeson High School for Business and Technology', 'Collocated school: Pathways in Technology Early College High School', 'K625', 'Performing Arts and Technology High School', '(Thomas Jefferson Educational Campus)', 'Poly Prep Country Day School', 'Progress High School for Professional Careers', 'Prospect Heights Educational Campus', 'See:', 'Prospect Heights High School (closed 2006)', 'Brooklyn Academy of Science and the Environment', 'Brooklyn School for Music & Theatre', 'The High School for Global Citizenship', 'International High School at Prospect Heights', '(split)', 'Rachel Carson High School for Coastal Studies', 'Roy Campanella Occupational Training Center', "St. Ann's School", 'St. Edmund Preparatory High School', 'St. Joseph High School', 'St. Saviour High School', 'Samuel J. Tilden Educational Campus', 'See:', 'Samuel J. Tilden High School (closed ~2010)', 'It Takes a Village Academy', 'Cultural Academy for the Arts and Sciences', 'The Kurt Hahn Expeditionary Learning School', '(split)', 'The School for Classics: An Academy of Thinkers, Writers and Performers', 'School for Democracy and Leadership', '(George W. Wingate Educational Campus)', 'The School for Human Rights', '(George W. Wingate Educational Campus)', 'School for International Studies', 'School for Legal Studies', 'Science Skills Center High School for Science, Technology and the Creative Arts', '(Waters Edge Educational Complex)', 'Science, Technology and Research Early College High School at Erasmus', '(Erasmus Hall Educational Campus)', 'Secondary School for Journalism', '(John Jay Educational Campus)', 'Secondary School for Law', '(John Jay Educational Campus)', 'Secondary School for Research (see Park Slope Collegiate)', '(renamed 2011)', 'Sheepshead Bay High School', 'Shulamith High School for Girls', 'Sinai Academy', 'Soille Bais Yaakov High School', 'South Brooklyn Community High School', 'South Shore Educational Campus', 'See:', 'South Shore High School (closed 2010)', 'Academy for Conservation and the Environment', 'Brooklyn Bridge Academy', 'Brooklyn Generation School', 'Brooklyn Theatre Arts High School', 'Victory Collegiate High School', 'K515', '(closing)', 'Sunset Park High School', 'Teachers Preparatory High School', 'Thomas Jefferson Educational Campus', 'See:', 'Thomas Jefferson High School (closed 2007)', 'FDNY High School for Fire and Life Safety', 'High School for Civil Rights', 'Performing Arts and Technology High School', 'World Academy for Total Community Health High School', '(split)', 'Tiferes Academy', 'Tiferes Bais Yaakov High School', 'Tiferes Miriam High School', 'Tomer Devora High School for Girls', 'Torah Academy High School', 'Transit Tech Career and Technical Education High School', 'United Lubavitcher Yeshiva', 'Urban Assembly Institute of Math and Science for Young Women', 'The Urban Assembly School for Law and Justice', 
             'The Urban Assembly School of Music and Art', '(Waters Edge Educational Complex)', 'Victory Collegiate High School', '(South Shore Educational Campus)', 'Waters Edge Educational Complex', 'See:', 'Brooklyn International High School', 'Science Skills Center High School for Science, Technology and the Creative Arts', 'Urban Assembly High School of Music and Art', '(collocated)', 'W.E.B. Dubois Academic High School', 'West Brooklyn Community High School', 'William E. Grady Career and Technical Education High School', 'William H. Maxwell Career and Technical Education High School', 'Williamsburg Charter High School', 'Williamsburg High School for Architecture and Design', '(Harry Van Arsdale Educational Campus)', 'Williamsburg Preparatory School', '(Harry Van Arsdale Educational Campus)', 'World Academy for Total Community Health High School', '(Thomas Jefferson Educational Campus)', 'Xaverian High School', 'Yeshiva Chanoch Lenaar', 'Yeshiva Derech HaTorah High School', 'Yeshiva Gedolah of Midwood', 'Yeshiva Gezdah Bais Yisrael', 'Yeshiva Karunas Halev', 'Yeshiva of Flatbush Joel Braverman High School', 'Yeshiva Ohr Moleh Koson', 'Yeshiva Sharro Yosher', 'Yeshiva Tiferes Shmiel', 'Yeshiva Toraf Hesed', 'Yeshiva Vyelipol School', 'Yeshivat Ateret Torah', 'Yeshivat Or Hatorah', 'Yeshivat Shaare Torah Boys High School', 'Yeshivat Shaare Torah Girls High School', "Yeshiva/Mesivta V'yoel Moshe School", 'Zvi Dov Roth Academy of Yeshiva Rambam', 'Manhattan', 'This section: viewtalkedit', 'School', 'NYC DOE', 'A. Philip Randolph Campus High School', 'Aaron School', 'Abraham Joshua Heschel School', 'Academy of Environmental Science Secondary High School', 'Academy for Social Action: A College Board School', 'Aichhorn School', 'The American Sign Language and English Secondary School', 'Avenues: The World School', 'Bard High School Early College', '(see also Bard High School Early College II in Queens)', 'Ballet Tech, NYC Public School for Dance', 'Baruch College Campus High School', 'Bayard Rustin Educational Complex', 'The Beacon School', '(Beacon High School)', 'The Beekman School', 'Bread & Roses Integrated Arts High School', 'Brearley School', 'The Browning School', 'The Calhoun School', 'Cascades High School', '(Cascades Center for Teaching & Learning)', 'Cathedral High School', 'Central Park East High School', 'Chapin School (Manhattan)', 'Chelsea Career and Technical Education High School)', 'Choir Academy of Harlem', 'City-As-School', '(High School M560 - City As School)', 'Coalition School for Social Change', 'Collegiate School', 'Columbia Grammar & Preparatory School', 'Columbia Secondary School', '(Columbia Secondary School for Math, Science & Engineering)', 'Cristo Rey New York High School', 'Convent of the Sacred Heart', 'The Dalton School', 'Daytop Preparatory School', 'Dominican Academy', 'The Dwight School', 'East Side Community High School', 'Edward A. Reynolds West Side High School', 'Eleanor Roosevelt High School', 'Elisabeth Irwin High School', '(Little Red Schoolhouse & Elisabeth Irwin High School) (LREI)', 'Essex Street Academy', '(Seward Park campus)', 'The Facing History School', '(Park West campus)', 'Fiorello H. LaGuardia High School of Music & Art and Performing Arts', 'Food and Finance High School', '(Park West campus)', 'Frederick Douglass Academy', 'See also:', 'Frederick Douglass Academy II Secondary School (Manhattan)', 'Frederick Douglass Academy III Secondary School (Bronx)', 'Frederick Douglass Academy IV Secondary School (Brooklyn)', 'Frederick Douglass Academy VI High School (Queens)', 'Frederick Douglass Academy VII High School (Brooklyn)', 'M499', 'Frederick Douglass Academy II Secondary School', 'Friends Seminary', 'Fusion Academy & Learning Center', 'George Washington High School', 'See:', 'High School for Health Careers and Sciences', 'High School for International Business and Finance', 'High School for Law and Public Service', 'High School for Media and Communications', '(split)', 'Gramercy Arts High School', 'Greenwich Village High School', 'Gregorio Luperon High School for Math & Science', '(Gregorio Luperon High School for Science and Mathematics)', 'Harlem Renaissance High School', 'Harvey Milk High School', 'Henry Street School for International Studies', 'The Heritage School', 'The Hewitt School', 'High School for Arts, Imagination and Inquiry', '(Martin Luther King, Jr. campus)', 'High School for Dual Language and Asian Studies', '(Seward Park campus)', 'High School For Environmental Studies', 'High School for Health Careers and Sciences', '(George Washington campus)', 'High School for Health Professions and Human Services', 'High School for Humanities Educational Campus', 'See:', 'Bayard Rustin High School for the Humanities', 'Humanities Preparatory Academy', 'The James Baldwin School', ' ', 'High School for International Business and Finance', '(George Washington campus)', 'High School for Law, Advocacy, and Community Justice', '(Martin Luther King, Jr. campus)', 'High School for Law and Public Service', '(George Washington campus)', 'High School for Leadership and Public Service', 'High School for Math, Science and Engineering at City College', 'High School for Media and Communications', '(George Washington campus)', 'High School of Art and Design', 'High School of Arts and Technology', '(Martin Luther King, Jr. campus)', 'High School of Economics and Finance', 'High School of Fashion Industries', 'High School of Graphic Communication Arts', 'Collocated school: Urban Assembly Gateway School for Technology', 'High School of Hospitality Management', '(Park West campus)', 'Humanities Preparatory Academy', '(High School for Humanities Educational Campus)', 'Hunter College High School', 'Independence High School', 'Institute for Collaborative Education', 'Jacqueline Kennedy Onassis High School', 'The James Baldwin School: A School for Expeditionary Learning', '(High School for Humanities Educational Campus)', 'John V. Lindsay Wildcat Academy Charter School', 'Julia Richman Education Complex', 'See:', 'Julia Richman High School (closed 1995)', 'Manhattan International High School', 'Talent Unlimited High School', 'Vanguard High School', '(split)', 'La Salle Academy', 'Landmark High School', 'Legacy School for Integrated Studies', 'Léman Manhattan Preparatory School', 'Liberty High School Academy for Newcomers', 'Life Sciences Secondary School', 'Louis D. Brandeis High School', 'Lower East Side Preparatory High School', 'Lower Manhattan Arts Academy', '(Seward Park campus)', 'Loyola School', 'Lycée français de New York (LFNY)', 'Manhattan Bridges High School', '(Park West campus)', 'Manhattan Center for Science and Mathematics', 'Manhattan Comprehensive Night and Day High School', 'Manhattan High School for Girls', 'Manhattan/Hunter College High School for Sciences', '(Manhattan/Hunter Science High School)', '(Hunter College High School for the Sciences)', '(Martin Luther King, Jr. campus)', 'Manhattan International High School', '(Julia Richman Education Complex - MIHS)', 'Manhattan Theatre Lab High School', '(Martin Luther King, Jr. campus)', 'Manhattan Village Academy', 'Marta Valle High School', '(Marta Valle Secondary School)', 'Martin Luther King High School', 'See:', 'High School for Arts, Imagination and Inquiry', 'High School for Law, Advocacy and Community Justice', 'High School of Arts and Technology', 'Hunter College High School (Manhattan/Hunter Science High School)', 'Manhattan Theatre Lab High School', 'The Urban Assembly School for Media Studies', '(split)', 'Marymount School', 'Millennium High School', 'Mother Cabrini High School', 'Mott Hall High School', 'Murry Bergtraum High School for Business Careers', 'New Design High School', '(Seward Park campus)', 'New Explorations into Science, Technology and Math High School (NEST+m High School)', 'NYC iSchool', 'New York City Lab School for Collaborative Studies', 'New York City Museum School', 'Nightingale-Bamford School', 'Norman Thomas High School', 'Northeastern Academy', 'Notre Dame School', 'Pace High School', 'Park East High School', 'Park West High School', 'See:', 'The Facing History School', 'Food and Finance High School', 'High School of Hospitality Management', 'Manhattan Bridges High School', 'The Urban Assembly School of Design and Construction', '(split)', "Professional Children's School", 'Professional Performing Arts School', 'Quest to Learn', 'Ramaz School', 'Rebecca School', 'Regis High School', 'Repertory Company High School for Theatre Arts', 'Rice High School', 'Richard R. Green High School of Teaching', 'The Robert Louis Stevenson School', 'St. Agnes Boys High School', 'St. Jean Baptiste High School', 'St. Michael Academy', 'St. Vincent Ferrer High School', 'Sacred Heart', '(See Convent of the Sacred Heart)', 'Satellite Academy High School', 'School for the Physical City High School', 'School of the Future', 'Seward Park High School', 'See:', 'Essex Street Academy', 'High School for Dual Language and Asian Studies', 'Lower Manhattan Arts Academy', 'New Design High School', 'The Urban Assembly Academy of Government and Law', '(split)', 'The Smith School', 'Solomon Schechter High School of New York', 'Spence School', 'Stuyvesant High School', 'Talent Unlimited High School', '(Julia Richman Education Complex - TUHS)', 'Thurgood Marshall Academy for Learning and Social Change', 'Trevor Day School', 'Trinity School', 'United Nations International School', 'Unity Center for Urban Technologies', 'University Neighborhood High School', 'Urban Academy Laboratory High School', '(Julia Richman Education Complex - UALHS)', 'The Urban Assembly Academy of Government and Law', '(Seward Park campus)', 'Urban Assembly Gateway School for Technology', 'Collocated at High School of Graphic Communication Arts', 'The Urban Assembly New York Harbor School', '(Governors Island)', 'The Urban Assembly School for Media Studies', '(Martin Luther King, Jr. campus)', 'Urban Assembly School for the Performing Arts'
             'The Urban Assembly School of Business for Young Women', 'The Urban Assembly School of Design and Construction', '(Park West campus)', 'Urban Peace Academy', 'Vanguard High School', '(Julia Richman Education Complex - VHS)', 'Wadleigh Secondary School for the Performing & Visual Arts', 'Washington Irving High School', 'Winston Preparatory School', 'Xavier High School', 'Yeshiva University High School for Boys', 'York Preparatory School', "Young Women's Leadership School", 'Washington Heights Expeditionary Learning School', 'Queens', 'This section: viewtalkedit', 'School', 'NYC DOE', 'Academy for Careers in Television and Film', 'Academy of American Studies', 'Academy of Finance and Enterprise', 'Academy of Medical Technology', '(Far Rockaway Educational Campus)', 'Al-Iman School', 'Archbishop Molloy High School', 'August Martin High School', 'Aviation High School', 'Q610', 'Baccalaureate School for Global Education', 'Bais Yaakov Machon Academy', 'Bard High School Early College II', '(see also Bard High School Early College in Manhattan)', 'Bayside High School', 'Beach Channel High School (closing 2014)', 'Collocated schools:', 'Channel View School for Research – Q262', 'Rockaway Collegiate High School – Q351', 'Rockaway Park High School for Environmental Sustainability – Q324', 'Q410', 'Cambria Heights Academy for New Literacies', 'Campus Magnet Educational Campus', 'Collocated schools:', 'Andrew Jackson High School (Queens) (closed 1994)', 'Humanities and the Arts Magnet High School – Q498', 'Mathematics, Science Research and Technology Magnet High School – Q492', 'Institute for Health Professions at Cambria Heights – Q243', 'Benjamin Franklin High School for Finance & Information Technology - Q313', 'Business, Computer Applications & Entrepreneurship High School (closed 2016)', 'Law, Government and Community Service High School (closed 2016)', 'Q490', 'Benjamin N. Cardozo High School', 'Cathedral Preparatory Seminary', 'Channel View School for Research', '(Beach Channel Educational Campus)', 'The Child School Legacy High School, Roosevelt Island', 'Christ The King Regional High School', 'East-West School of International Studies', 'Evangel Christian School', 'Excelsior Preparatory High School', '(Springfield Gardens Educational Campus)', 'Ezra Academy', 'Far Rockaway Educational Campus', 'See:', 'Far Rockaway High School – Q465 (closed 2011)', 'Academy of Medical Technology: A College Board School – Q309', 'Frederick Douglass Academy VI High School – Q260', 'Queens High School for Information, Research, and Technology – Q302', '(split)', 'Flushing High School', 'Flushing International High School', 'Forest Hills High School', 'Francis Lewis High School', 'Frank Sinatra School of the Arts High School', 'Frederick Douglass Academy VI High School', '(Far Rockaway Educational Campus)', 'Garden School', 'Gateway to Health Science High School', 'George Washington Carver High School for the Sciences', '(Springfield Gardens Educational Campus)', 'Greater New York Academy of Seventh-day Adventists', 'Grover Cleveland High School', 'High School for Arts and Business', 'High School for Community Leadership', '(Jamaica High School campus)', 'High School for Construction Trades, Engineering and Architecture', 'High School for Law Enforcement and Public Safety', 'High School of Applied Communication', 'Hillcrest High School', 'Hillside Arts & Letters Academy', '(Jamaica High School campus)', 'Holy Cross High School', 'Humanities & Arts Magnet High School', 'Information Technology High School', 'International High School at LaGuardia Community College', 'Jamaica Gateway to the Sciences', '(Jamaica High School campus)', 'Jamaica High School', 'Collocated schools:', 'High School for Community Leadership – Q328', 'Hillside Arts & Letters Academy – Q325', 'Jamaica Gateway to the Sciences – Q350', 'Queens Collegiate: A College Board School – Q310', 'Q470', 'John Adams High School', 'John Bowne High School', 'John F. Kennedy Jr. School', '(formerly Queens Occupational Training Center)', 'Long Island City High School', 'The Lowell School', 'Martin Luther High School', 'Martin Van Buren High School', 'The Mary Louis Academy', 'Maspeth High School', 'Mathematics, Science Research and Technology Magnet High School', 'Mesivta Ohr Torah School', 'Mesivta Yesodei Yeshurun', 'Metropolitan Expeditionary Learning School', '(Queens Metropolitan High School campus)', 'Middle College High School at LaGuardia Community College', 'Monsignor McClancy Memorial High School', 'Newcomers High School', 'Newtown High School', 'North Queens Community High School', 'Pan American International High School', 'Pathways College Preparatory School: A College Board School', 'Preparatory Academy for Writers: A College Board School', '(Springfield Gardens Educational Campus)', 'Project Blend School, Jamaica', 'Queens Academy High School', 'Queens Collegiate: A College Board School', '(Jamaica High School campus)', 'Queens Gateway to Health Sciences Secondary School', 'Queens High School for Information, Research, and Technology', '(Far Rockaway Educational Campus)', 'Queens High School for the Sciences at York College', 'Queens High School of Teaching, Liberal Arts and the Sciences', 'Queens Metropolitan High School', 'Collocated schools:', 'Metropolitan Expeditionary Learning High School – Q167', 'Q686', 'Queens Preparatory Academy', '(Springfield Gardens Educational Campus)', 'Queens Vocational and Technical High School', 'Razi School', 'The Renaissance Charter School', 'Richmond Hill High School', 'Robert F. Kennedy Community High School', 'Robert F. Wagner, Jr. Secondary School for Arts and Technology', '(Robert F. Wagner, Jr. Institute for Arts & Technology)', 'Robert H. Goddard High School of Communication Arts and Technology', 'Rockaway Collegiate High School', '(Beach Channel Educational Campus)', 'Rockaway Park High School for Environmental Sustainability', '(Beach Channel Educational Campus)', 'St. Agnes High School', 'St. Demetrios School', 'St. Francis Preparatory School', 'St. George Academy', "St. John's Preparatory School", "Scholars' Academy", 'Shevach High School', 'Springfield Gardens Educational Campus, includes:', 'Excelsior Preparatory High School', 'George Washington Carver High School for the Sciences', 'Preparatory Academy for Writers: A College Board School', 'Queens Preparatory Academy', ' ', 'Stella Maris High School (closing June 2010)', 'Summit School', 'Thomas A. Edison Vocational and Technical High School', 'Torah Academy High School for Girls', 'Townsend Harris High School', 'VOYAGES Preparatory', 'Whitestone Academy', 'William Cullen Bryant High School', 'Windsor School', 'World Journalism Preparatory: A College Board School', 'Yeshiva Berachel David-Torah School', 'Yeshiva Binat Chaim - Boys School', 'Yeshiva of Far Rockaway', '(Yeshiva Derech Ayson)', 'Yeshiva Shaar Hatoreh Research', 'Yeshiva University High School for Girls', '(Samuel H. Wang Yeshiva University High School for Girls)', "Young Women's Leadership School, Astoria", "Young Women's Leadership School, Queens", 'Staten Island', 'This section: viewtalkedit', 'School', 'NYC DOE', 'College of Staten Island High School for International Studies', '(CSI High School for International Studies)', 'Concord High School', 'Curtis High School', 'Gaynor McCown Expeditionary Learning School', 'Mesivta Kesser Yisroel', 'Michael J. Petrides School', 'Miraj Islamic School', 'Monsignor Farrell High School', 'Moore Catholic High School', 'New Dorp High School', 'Notre Dame Academy High School', 'Port Richmond High School', 'Ralph R. McKee Career and Technical Education High School', 'Richard H. Hungerford School', 'St. Joseph by the Sea High School', 'St. Joseph Hill Academy', "St. Peter's Boys High School", 'Staten Island Academy', 'Staten Island Technical High School', 'Susan E. Wagner High School', 'Tottenville High School', 'Yeshiva of Staten Island', 'Yeshiva Tifereth Torah']

    if type(s)!=str or s==None:
        return 0
    school=0
    for school in schools:
        ratio=fuzz.ratio(school.lower(), s.lower())
        if ratio>=85:
            school=1
    return school

In [16]:
def check_car_make(value):
    car_make = ['BMW', 'TOYOT', 'AUDI', 'NISSA', 'DODGE', 'ACURA', 'HYUND', 'MINI', 'LEXUS', 'SUBAR', 
                'HONDA', 'KIA', 'CHEVR', 'FORD', 'BUICK', 'VOLKS', 'GMC', 'VOLVO', 'INFIN', 'JEEP',
                'ACADE','AERO','AJAX','ALEXA','ALFAR','AMC','ANCIA','APRIL','AS/M','AUSTI','AUTOC','BENTL',
                'BERTO','BL/B','BL/BI','BRIDG','CADIL','CARGO','CHECK','CHRYS','DORSE','DUCAT','FERRA',
                'FIAT','FRUEH','HARLE','HIN','HINO','HUMME','INTER','ISUZU','JAGUA','KAWAS','KENWO',
                'LINCO','MACK','MASSA','MAZDA','MERCU','MINI','MITSU','NS/OT']

    if type(value)!=str or value==None:
        return 0
    is_car_make=0
    for brand in car_make:
        ratio=fuzz.partial_ratio(brand.lower(), value.lower())
        if ratio>=85:
            is_car_make=1
    return is_car_make

In [17]:
def check_school_level(s):
    school_level = ['K-8','K-2','K-3' 'Elementary', 'Elementary School', 'Middle', 'Middle School', 'Transfer', 'High',
                    'High School','D75','YABC']
    
    if type(s)!=str or s==None:
        return 0
    is_lvl=0
    for lvl in school_level:
        ratio=fuzz.ratio(lvl.lower(), s.lower())
        if ratio>=85:
            is_lvl=1
    
    return is_lvl

In [18]:
def check_borough(s):
    borough_list = ['brooklyn', 'bronx', 'queens', 'manhattan', 'staten island']
    county=['K','M','Q','R','X']
    if type(s)!=str or s==None:
        return 0
    if len(s)==1:
        if s.upper() in county:
            return 1
    is_boro=0
    for boro in borough_list:
        ratio=fuzz.ratio(s.lower(), boro.lower())
        if ratio>=85:
            is_boro=1
    return is_boro    

In [19]:
import re
def check_zip_code(value):
    if value==None:
        return 0
    if type(value)!=str:
        value=str(value)
    regex = re.compile(r"(^[0-9]{5}(?:-[0-9]{4})?$)")
    if re.fullmatch(regex,value.lower()) != None:
        return 1
    else:
        return 0

In [20]:
def check_neighborhood(value):
    if type(value)!=str or value==None:
        return 0
    manhattan = ['Alphabet','Battery Park','Carnegie Hill','Chelsea','Chinatown','East Harlem','East Village',\
                'Financial District','Flatiron District','Gramercy Park','Greenwich Village','Harlem','Clinton','Inwood',\
                'Kips Bay','Lincoln Square','Lower East Side','Manhattan Valley','Midtown East','Midtown West',\
                'Morningside Heights','Murray Hill','NoLita','Little Italy','Roosevelt Island','SoHo','Tribeca','Upper East Side',\
                'Upper West Side','Washington Heights','West Village']
    bronx =['Baychester','Bedford Park','Belmont','Bronxdale','Castle Hill','City Island','Concourse Village',\
            'Grand Concourse','Morrisania','Country Club','Fieldston','Fordham','Hunts Point','Kingsbridge','Kingsbridge Heights',\
            'Melrose','Morris Heights','Morris Park','Mott Haven','Parkchester','Pelham Bay','Pelham Gardens','Pelham Parkway',\
            'Port Morris','Riverdale','Soundview','Throgs Neck','Tremont','University Heights','Wakefield','Williamsbridge','Woodlawn']
    brooklyn =['Bath Beach','Bay Ridge','Bedford-Stuyvesant','Bensonhurst','Bergen Beach','Boerum Hill','Borough Park',\
            'Brighton Beach','Brooklyn Heights','Brownsville','Bushwick','Canarsie','Carroll Gardens','Cobble Hill','Coney Island',\
            'Crown Heights','Cypress Hills','Downtown Brooklyn','Dumbo','Vinegar Hill','Dyker Heights','East New York','Flatbush',\
            'Flatlands','Fort Greene','Clinton Hill','Gerritsen Beach','Gowanus','Gravesend','Greenpoint','Greenwood Heights',\
            'Manhattan Beach','Marine Park','Midwood','Mill Basin','Park Slope','Prospect Heights','Prospect Park South','Kensington',\
            'Red Hook','Sea Gate','Sheepshead Bay','Sunset Park','Williamsburg','Windsor Terrace']
    queens = ['Arverne','Astoria','Bayside','Beechhurst','Belle Harbor','Neponsit','Bellerose','Briarwood','Broad Channel','Cambria Heights',\
            'College Point','Corona','Douglaston','East Elmhurst','Elmhurst','Far Rockaway','Floral Park','Flushing','Forest Hills',\
            'Fresh Meadows','Glen Oaks','Glendale','Hillcrest','Hollis','Hollis Hills','Howard Beach','Jackson Heights','Jamaica',\
            'Jamaica Estates','Jamaica Hills','Kew Gardens','Laurelton','Little Neck','Long Island City','Maspeth','Middle Village',\
            'Oakland Gardens','Ozone Park','Queens Village','Rego Park','Richmond Hill','Ridgewood','Rockaway Park','Rosedale',\
            'South Jamaica','South Ozone Park','Springfield Gardens','St. Albans','Sunnyside','Whitestone','Woodhaven','Woodside']
    staten = ['Annadale','Arden Heights','Arrochar','Bay Street','Bulls Head','Castleton Corners','Charleston','Clifton','Dongan Hills',\
            'Eltingville','Emerson Hill','Graniteville','Grant City','Grasmere','Concord','Great Kills','Grymes Hill','Huguenot',\
            'Livingston','Manor Heights','Mariners Harbor','Midland Beach','New Brighton','New Dorp','New Dorp Beach','New Springville',\
            'Oakwood','Pleasant Plains','Port Richmond','Prince Bay','Richmondtown','Rosebank','Rossville','Shore Acres','Silver Lake',\
            'South Beach','St. George','Stapleton','Sunnyside','Todt Hill','Tompkinsville','Tottenville','Travis','West New Brighton',\
            'Westerleigh','Willowbrook','Woodrow']
    all_neighborhoods = []
    all_neighborhoods.extend(manhattan)
    all_neighborhoods.extend(bronx)
    all_neighborhoods.extend(brooklyn)
    all_neighborhoods.extend(queens)
    all_neighborhoods.extend(staten)
    for neighborhood in all_neighborhoods:
        if fuzz.ratio(neighborhood.lower(),value.lower())>=85:
            return 1
    return 0

In [21]:
def check_address(value):
    if type(value)!=str or value==None:
        return 0
    street_name = ['street','st.','avenue','boulevard', 'pl', 'rd', 'alley', 'ave', 'bridge', 'blvd', 'ln', 'dr',
                  'slip', 'broaway', 'terrace', 'plaza', 'square', 'expy']
    for street in street_name:
        if fuzz.partial_ratio(street,value.lower())>=85 and value[0].isdigit() == True:
            return 1
    return 0

def check_street_name(value):
    if type(value)!=str or value==None:
        return 0
    street_name = ['street','st.','avenue','boulevard', 'pl', 'rd', 'alley', 'ave', 'bridge', 'blvd', 'ln', 'dr',
                  'slip', 'broaway', 'terrace', 'plaza', 'square', 'expy']
    for street in street_name:
        if fuzz.partial_ratio(street,value.lower())>=85 and value[0].isdigit() == False:
            return 1
    return 0

In [22]:
def check_city(value):
    if type(value)!=str or value==None:
        return 0
    city_list = ['Albany', 'Amsterdam', 'Auburn', 'Batavia', 'Beacon', 'Binghamton', 'Buffalo', 'Canandaigua', 'Cohoes', 'Corning', 'Cortland', 'Dunkirk', 'Elmira', 'Fulton', 'Geneva', 'Seneca[A]', '3609928640', 'Glen Cove', 'Glens Falls', 'Gloversville', 'Hornell', 'Hudson', 'Ithaca', 'Jamestown', 'Johnstown', 'Kingston', 'Lackawanna', 'Little Falls', 'Lockport', 'Long Beach', 'Mechanicville', 'Middletown', 'Mount Vernon', 'New Rochelle', 'New York', 'Newburgh', 'Niagara Falls', 'North Tonawanda', 'Norwich', 'Ogdensburg', 'Olean', 'Oneida', 'Oneonta', 'Oswego', 'Peekskill', 'Plattsburgh', 'Port Jervis', 'Poughkeepsie', 'Rensselaer', 'Rochester', 'Rome', 'Rye', 'Salamanca', 'Saratoga Springs', 'Schenectady', 'Sherrill', 'Syracuse', 'Tonawanda', 'Troy', 'Utica', 'Watertown', 'Watervliet', 'White Plains', 'Yonkers']
    for city in city_list:
        if fuzz.ratio(city,value.lower())>=85:
            return 1
    return 0

In [23]:
import re
def isValidPhoneNum(phoneNum):
    if phoneNum==None or type(phoneNum)!=int or type(phoneNum)!=str:
        return 0
    if type(phoneNum)==int:
        phoneNum=str(phoneNum)
    phoneNum = phoneNum.replace('-','')
    if re.search(r"((\(\d{3}\)?)|(\d{3}))([\s./]?)(\d{3})([\s./]?)(\d{4})", phoneNum):
        return 1
    else:
        return 0

In [24]:
def is_business(s):
    if type(s)!=str or s==None:
        return 0
    
    
    business_suffix = ['inc','inc.','llc','l.l.c.','p.c.','pc','lp','l.p.','corp','corp.','corporation','company','co.','co','ltd','ltd.','capital','holdings','services']
    business=0
    for suff in business_suffix:
        ratio=fuzz.ratio(suff, s.lower())
        if ratio>=90:
            business=1
    if re.search(r"^(?!\s)(?!.*\s$)(?=.*[a-zA-Z0-9])[a-zA-Z0-9 '~?!]{2,}$", s):
        business=1
    return business

In [25]:
def is_person_name(s):
    if type(s)!=str or s==None:
        return 0
    if re.search(r"^[a-zA-Z]+(([',. -][a-zA-Z ])?[a-zA-Z]*)*$",s):
        return 1
    return 0

In [26]:
def check_color(value):
    if type(value)!=str or value==None:
        return 0
    color_list = ['white', 'black', 'pink', 'purple', 'blue', 'gray', 'green', 'beige', 'yellow', 'ivory', 'gold', 
                  'orange', 'red', 'brown', 'BK','WH','GR','RD','BL','GY','SL']
    
    for color in color_list:
        ratio=fuzz.ratio(color.lower(), value.lower())
        if ratio>=95:
            return 1
        
    return 0 

In [27]:
import re
def check_LAT_LON_coordinates(value):
    try:
        lag = value.split(',')[0][1:-1]
        s = r'^[\-\+]?(0(\.\d{1,10})?|([1-9](\d)?)(\.\d{1,10})?|1[0-7]\d{1}(\.\d{1,10})?|180\.0{1,10})$'  # longitude
        if re.match(s, lag) != None:
            return 1
        else:
            return 0
    except:
        return 0


In [28]:
def semantic_type(x):
    if isValidPhoneNum(x)==1:
        return 'phone_number'
    elif check_zip_code(x)==1:
        return 'zip_code'
    elif check_borough(x)==1:
        return 'borough'
    elif check_LAT_LON_coordinates(x)==1:
        return 'lat_lon_cord'
    elif check_city(x)==1:
        return 'city'
    
    elif check_color(x)==1:
        return 'color'
    elif process_website(x)==1:
        return 'website'
    elif check_school_level(x)==1:
        return 'school_level'
    elif school_subject(x)==1:
        return 'subject_in_school'
    
    elif is_major(x)==1:
        return 'area_of_study'
    
    elif is_location_type(x)==1:
        return 'location_type'
    elif is_college(x)==1:
        return 'college_name'
    elif check_neighborhood(x)==1:
        return 'neighborhood'
    elif is_park(x)==1:
        return 'park_playground'
    elif is_school(x)==1:
        return 'school_name'
    elif check_street_name(x)==1:
        return 'street_name'
    elif check_address(x)==1:
        return 'address'
    elif check_car_make(x)==1:
        return 'car_make'
    elif is_building_classification(x)==1:
        return 'building_classification'
    elif is_vehicle_type(x)==1:
        return 'vehicle_type'
    elif is_agency(x)==1:
        return 'city_agency'
    elif is_business(x)==1:
        return 'business_name'
    elif is_person_name(x)==1:
        return 'person_name'
    else:
        return 'other'

In [29]:
if not os.path.exists('Task-2_Results_JSON'):
    os.makedirs('Task-2_Results_JSON')

In [30]:
given_label_list=["person_name", "business_name", "phone_number", "address", "street_name",
"city", "neighborhood", "lat_lon_cord", "zip_code", "borough", "school_name",
"color", "car_make", "city_agency", "area_of_study", "subject_in_school",
"school_level", "college_name", "website", "building_classification",
"vehicle_type", "location_type", "park_playground", "other"]

all_col_counts=""
for i,x in enumerate(contents):
    print(i,x)
    col_name=x.split('.')[1]
    x=x.replace(" ","")
    fp='NYCColumns/'+x
    df=dd.read_csv(fp, sep="\t", names=[x.split('.')[1], 'y'])
    df.drop('y', inplace=True, axis=1)
    df['Identified_Labels']=df[col_name].apply(lambda x: semantic_type(x))
    all_col_counts=all_col_counts+str(df['Identified_Labels'].count())+','
    pred_labels= list(set(list(df['Identified_Labels'])))
    
    json_dict={"column_name": col_name, "semantic_types":[]}
    
    for label in pred_labels:
        if label in given_label_list and label!='other':
            semantic_dict={"semantic_type":label, "count": int((df['Identified_Labels']==label).sum())}
            json_dict['semantic_types'].append(semantic_dict)
        else:
            semantic_dict={"semantic_type":label,"label":"" ,"count": int((df['Identified_Labels']==label).sum())}
            json_dict['semantic_types'].append(semantic_dict)
    json_file_path='Task-2_Results_JSON/'+x.split('.')[0]+'.json'
    print(json_dict)
    print('\n\n')
    with open(json_file_path, 'w', newline='\n') as json_file:
        json.dump(json_dict, json_file)

f=open("all_Col_counts.txt", 'w+')
f.write(all_col_counts)

0 s3k6-pzi2.website.txt.gz
{'column_name': 'website', 'semantic_types': [{'semantic_type': 'website', 'count': 438}]}



1  mrxb-9w9v.BOROUGH___COMMUNITY.txt.gz
{'column_name': 'BOROUGH___COMMUNITY', 'semantic_types': [{'semantic_type': 'building_classification', 'count': 1}, {'semantic_type': 'borough', 'count': 5}]}



2  crc3-tcnm.CORE_SUBJECT.txt.gz
{'column_name': 'CORE_SUBJECT', 'semantic_types': [{'semantic_type': 'subject_in_school', 'count': 3}, {'semantic_type': 'building_classification', 'count': 1}, {'semantic_type': 'vehicle_type', 'count': 1}]}



3  jhjm-vsp8.Agency.txt.gz
{'column_name': 'Agency', 'semantic_types': [{'semantic_type': 'vehicle_type', 'count': 37}, {'semantic_type': 'street_name', 'count': 3}, {'semantic_type': 'building_classification', 'count': 3}, {'semantic_type': 'lat_lon_cord', 'count': 1}]}



4  3rfa-3xsf.Incident_Zip.txt.gz
{'column_name': 'Incident_Zip', 'semantic_types': [{'semantic_type': 'building_classification', 'count': 3}, {'semantic_type

1113